In [1]:
# Set your OpenAI API key (required for openai/gpt-4o-mini).
# When you run this cell, a prompt will appear—paste your key there (it won't be shown on screen).
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    key = getpass.getpass("OpenAI API key: ")
    if key:
        os.environ["OPENAI_API_KEY"] = key
    else:
        print("No key entered. Set OPENAI_API_KEY in this notebook or in your shell.")

In [2]:
import dspy

lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)

/Users/eddiej@nisos.com/mespace/dspy/dspy-venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [21]:

qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

print(response.response)

In Linux, "high memory" and "low memory" refer to different regions of the system's memory address space, particularly in the context of how the kernel manages memory for processes.

- **Low Memory**: This typically refers to the memory that is directly accessible by the kernel and can be used by processes without any special handling. In a 32-bit system, this is usually the first 896 MB of RAM (though this can vary based on the architecture and configuration). Low memory is used for kernel data structures and for user-space processes that require direct access to memory.

- **High Memory**: This refers to memory that is above the low memory limit and is not directly accessible by the kernel in a 32-bit system. Processes can use this memory, but the kernel must use special mechanisms (like paging) to access it. High memory is often used in systems with large amounts of RAM, allowing more memory to be allocated to user-space processes while still maintaining a limited address space for 

In [22]:
# Use Tuple[str, str] — string signatures don't support tuple(str, str)
timer = dspy.Predict('flight_time: Tuple[str, str] -> duration_in_secs: int')
response = timer(flight_time=("12:00", "13:09"))
print(response)

Prediction(
    duration_in_secs=414
)


You can inspect the n last prompts sent by DSPy easily. Alternatively, if you enabled MLflow Tracing above, you can see the full LLM interactions for each program execution in a tree view.

In [23]:
dspy.inspect_history(n=1)





[2026-02-13T19:48:30.222458]

System message:

Your input fields are:
1. `flight_time` (tuple[str, str]):
Your output fields are:
1. `duration_in_secs` (int):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## flight_time ## ]]
{flight_time}

[[ ## duration_in_secs ## ]]
{duration_in_secs}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `flight_time`, produce the fields `duration_in_secs`.


User message:

[[ ## flight_time ## ]]
["12:00", "13:09"]

Respond with the corresponding output fields, starting with the field `[[ ## duration_in_secs ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## duration_in_secs ## ]]
414

[[ ## completed ## ]]







DSPy has various built-in modules, e.g. dspy.ChainOfThought, dspy.ProgramOfThought, and dspy.ReAct. These are interchangeable with basic dspy.Predict: they take your signature, which is specific to your task, and they apply general-purpose prompting techniques and inference-time strategies to it.

For example, dspy.ChainOfThought is an easy way to elicit reasoning out of your LM before it commits to the outputs requested in your signature.

In the example below, we'll omit str types (as the default type is string). You should feel free to experiment with other fields and types, e.g. try topics: list[str] or is_realistic: bool.


In [19]:
# cot = dspy.ChainOfThought('question -> response: list[str]')
# cot(question="should curly braces appear on their own line?")

# cot = dspy.ChainOfThought('question -> response: bool')
# cot(question="should curly braces appear on their own line?")

cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='The placement of curly braces on their own line is often a matter of coding style and conventions. In many programming languages, such as Java, C#, and JavaScript, it is common to place opening curly braces on the same line as the preceding statement, while closing curly braces may be placed on their own line. This style enhances readability and maintains a clean structure. However, some coding standards, like those used in Python or certain configurations of C++, may prefer braces to be on their own lines for clarity. Ultimately, it depends on the specific coding guidelines being followed.',
    response="Curly braces do not necessarily have to appear on their own line; it depends on the coding style and conventions being used. Many developers prefer to keep opening braces on the same line as the preceding statement for readability, while closing braces are often placed on their own line. It's best to follow the coding standards of the project or language yo

In [24]:
cot_flight = dspy.ChainOfThought('flight_time: Tuple[str, str] -> duration_in_secs: int')
response = cot_flight(flight_time=("12:00", "13:09"))

In [25]:
dspy.inspect_history(n=2)





[2026-02-13T19:48:30.222458]

System message:

Your input fields are:
1. `flight_time` (tuple[str, str]):
Your output fields are:
1. `duration_in_secs` (int):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## flight_time ## ]]
{flight_time}

[[ ## duration_in_secs ## ]]
{duration_in_secs}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `flight_time`, produce the fields `duration_in_secs`.


User message:

[[ ## flight_time ## ]]
["12:00", "13:09"]

Respond with the corresponding output fields, starting with the field `[[ ## duration_in_secs ## ]]` (must be formatted as a valid Python int), and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## duration_in_secs ## ]]
414

[[ ## completed ## ]]





[2026-02-13T19:52:28.323523]

System message:

Your input fields are:
1. `flight_time` (tuple[s